In [5]:
"""
Pedro Tobarra 20210810:

Modificacion del notebook '20210524 alerta impago nomina pension desempleo.ipynb' para meter el codigo en una funcion de
python que luego pueda ser implementada en el fichero 'impago_nomina_pension_desempleo_20210607.py' para su integracion con el
backend del PFM
"""

"\nPedro Tobarra 20210810:\n\nModificacion del notebook '20210524 alerta impago nomina pension desempleo.ipynb' para meter el codigo en una funcion de\npython que luego pueda ser implementada en el fichero 'impago_nomina_pension_desempleo_20210607.py' para su integracion con el\nbackend del PFM\n"

In [6]:
from pandas import read_csv, read_excel, date_range, RangeIndex, DataFrame, to_datetime
from datetime import datetime, timedelta
from matplotlib import pyplot
import statistics
from pandas.tseries.offsets import MonthEnd
import calendar

In [7]:
# Load data using read_excel
transacciones_nomina_orig_df = read_excel('20210513 mmelero (249236).xlsx', sheet_name='Hoja1')
transacciones_nomina_orig_df

,Fecha transacción,Importe,ID Categoría,Nombre categoría,Nivel categoría,iD categoría padre,Nombre categoría padre,Proveedor,Marca,Sector,Financiero,Transferencia,Unnamed: 12,Unnamed: 13,ID,BALANCE,BALANCE_DATE
0,2020-07-24,-1.20,84.0,parking_84,2.0,5.0,vehiculos_5,NaN,NaN,NaN,False,False,NaN,NaN,249236.0,1478.98,2020-07-26
1,2020-07-23,-4.50,117.0,regalos_celebraciones_117,2.0,8.0,gastos_personales_8,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaT
2,2020-07-23,-13.53,70.0,supermercados_70,2.0,4.0,alimentacion_4,SPAR,SPAR,Supermercados,False,False,NaN,NaN,NaN,NaN,NaT
3,2020-07-22,-51.40,222.0,adsl_222,3.0,521.0,Comunicaciones y TV,Jazz Telecom,Jazztel,Compañías telecomunicaciones,False,False,NaN,NaN,NaN,NaN,NaT
4,2020-07-22,-11.00,70.0,supermercados_70,2.0,4.0,alimentacion_4,Mercadona,Mercadona,Supermercados,False,False,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,2016-09-05,1533.02,18.0,retribucion_liquida_18,3.0,576.0,Nómina,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,NaT
1483,2016-09-05,-617.47,126.0,liquidacion_tarjeta_126,3.0,478.0,tarjetas_credito_478,NaN,NaN,NaN,True,True,NaN,NaN,NaN,NaN,NaT
1484,2016-09-05,-138.08,125.0,cuotas_125,3.0,127.0,otros_pasivos_127,NaN,NaN,NaN,True,False,NaN,NaN,NaN,NaN,NaT
1485,2016-09-01,-32.22,90.0,seguro_salud_90,2.0,6.0,salud_6,Sanitas S.A. de Seguros,Sanitas,Seguros de salud,True,False,NaN,NaN,NaN,NaN,NaT


In [8]:
SALARIO = 950.0
PENSION = 609.9
DESEMPLEO = 451.92


def impago_nomina_pension_desempleo(transacciones_nomina_df):

    cobrado = False

    # nos quedamos con transacciones desde 'Fecha transacción' hasta 'ID Categoría'
    transacciones_nomina_df = transacciones_nomina_df.iloc[:, 0:3]

    """
    nos quedamos con las transacciones que cumplan las siguientes condiciones:
    (18.0) retribucion_liquida_18 >= 950.0 eur OR
    (588.0) Prestación de Seguridad Social >= 609.9 eur OR
    (589.0) Prestación de desempleo >= 451.92 eur
    """
    transacciones_nomina_df = \
        transacciones_nomina_df[
            (transacciones_nomina_df['ID Categoría'] == 18.0) & (transacciones_nomina_df['Importe'] >= SALARIO) |
            (transacciones_nomina_df['ID Categoría'] == 588.0) & (transacciones_nomina_df['Importe'] >= PENSION) |
            (transacciones_nomina_df['ID Categoría'] == 589.0) & (transacciones_nomina_df['Importe'] >= DESEMPLEO)]

    # borramos la columna de 'ID Categoría'
    transacciones_nomina_df = transacciones_nomina_df.drop(['ID Categoría'], axis=1)

    # ordenamos las filas del dataframe por fechas en orden ascendente
    transacciones_nomina_df.sort_values(by='Fecha transacción', inplace=True)

    # renombramos columnas
    transacciones_nomina_df.columns = ['FECHA', 'IMPORTE']

    # agrupamos los valores por FECHA para quitarnos los duplicados
    transacciones_nomina_df = transacciones_nomina_df.groupby(transacciones_nomina_df.FECHA).sum()

    # hacemos un dataframe con el dia que pagaron la nomina para ver en que dias se cobra mas frecuentemente la misma

    # hacemos una columna con la fecha a partir del índice
    transacciones_nomina_df['FECHA'] = transacciones_nomina_df.index

    # hacemos una columna con el dia a partir de la columna de la fecha
    transacciones_nomina_df['DIA'] = transacciones_nomina_df['FECHA'].dt.day

    # calculamos la moda
    stat_mode = statistics.mode(transacciones_nomina_df['DIA'])

    # extraemos el 1er cuartil
    quartil1_dist = int(transacciones_nomina_df.describe().loc['25%']['DIA'])

    # extraemos el 3er cuartil
    quartil3_dist = int(transacciones_nomina_df.describe().loc['75%']['DIA'])

    iqr_dist = quartil3_dist - quartil1_dist

    # si NO cobramos a FIN DE MES
    if stat_mode < 28:
        iqr = iqr_dist
    # si SÍ cobramos a FIN DE MES
    else:
        # si quartil3_dist - quartil1_dist es mayor a 4 dias
        if iqr_dist > 4:
            iqr = 4
        else:
            iqr = iqr_dist

    # pedimos al usuario la fecha (en producción tomamos la fecha del sistema)
    #     year = input('year: ')
    #     month = input('month: ')
    #     day = input('day: ')
    year = str(2020)
    month = str(8)
    day = str(31)

    # pasamos la fecha a string
    current_date_str = year + '-' + month + '-' + day

    current_date_obj = datetime.strptime(current_date_str, '%Y-%m-%d')

    # pasamos iqr a formato datetime
    iqr_obj = timedelta(days=iqr)

    # calculamos quartil3_obj en funcion de current_date_obj, quartil3_dist y si cobramos a FIN de MES o NO

    # si SÍ cobramos a FIN de MES
    if stat_mode >= 28:
        # quartil3_obj sera el ultimo dia del mes de current_date_obj
        quartil3 = calendar.monthrange(current_date_obj.year, current_date_obj.month)[1]
        quartil3_str = str(current_date_obj.year) + '-' + str(current_date_obj.month) + '-' + str(quartil3)
    # si NO cobramos a FIN de MES
    else:
        quartil3_str = str(current_date_obj.year) + '-' + str(current_date_obj.month) + '-' + str(quartil3_dist)

    q3_obj = datetime.strptime(quartil3_str, '%Y-%m-%d')

    q1_obj = q3_obj - iqr_obj

    lim_inf_obj = q1_obj - 1.5 * iqr_obj
    # quitamos partes de dias no enteros que haya podido introducir 1.5*iqr_obj
    lim_inf_obj = lim_inf_obj.replace(hour=0, minute=0, second=0, microsecond=0)

    # calculamos limite superior de barrido segun cobremos a fin de mes o no
    lim_sup_obj = q3_obj + timedelta(days=1)

    for d in range(int((lim_sup_obj - lim_inf_obj).days)):
        # ESTA LINEA NO SE EJECUTA EN PRODUCCIÓN
        # fecha_str = str((lim_inf_obj + timedelta(d)).year) + '-' + str((
        # lim_inf_obj + timedelta(d)).month) + '-' + str((lim_inf_obj + timedelta(days=d)).day)
        # ESTA LINEA NO SE EJECUTA EN PRODUCCIÓN
        # print(fecha_str)
        if (lim_inf_obj + timedelta(days=d)) in transacciones_nomina_df.index:
            cobrado = True
            mensaje = 'El usuario cobró una nómina de ' + \
                      str(transacciones_nomina_df['IMPORTE'][str(lim_inf_obj + timedelta(days=d))[:10]]) + \
                      ' eur el día ' + str(lim_inf_obj + timedelta(d))[:10]
    #             print ('El usuario cobró una nómina de ' + \
    #                    str(transacciones_nomina_df['IMPORTE'][str(lim_inf_obj + timedelta(days=d))[:10]]) + \
    #                    ' eur el día ' + str(lim_inf_obj + timedelta(d))[:10])
    if not cobrado:
        fecha_obj = lim_sup_obj - timedelta(days=1)
        fecha = str(fecha_obj.year) + '-' + str(fecha_obj.month) + '-' + str(fecha_obj.day)
        mensaje = fecha + ': El usuario aún no ha cobrado la nómina. Hay que avisarle'
    #         print(fecha + ': El usuario aún no ha cobrado la nómina. Hay que avisarle')

    # if para decidir si genero aviso

    if not cobrado and current_date_obj == q3_obj:
        aviso = True
    else:
        aviso = False

    # ESTA LINEA NO SE EJECUTA EN PRODUCCIÓN
    #     print('Aviso: ' + str(aviso))

    #     print(lim_sup_obj)
    #     print(lim_inf_obj)
    #     print(q1_obj)
    #     print(q3_obj)
    #     print(quartil3_str)
    #     print(iqr_obj)
    #     print(current_date_obj)
    #     print(current_date_str)
    #     print(iqr)
    #     print(iqr_dist)
    #     print(quartil3_dist)
    #     print(quartil1_dist)
    #     print(stat_mode)
    #     print(transacciones_nomina_df)

    return aviso, mensaje


# Load data using read_excel
transacciones_nomina_orig_df = read_excel('20210513 mmelero (249236).xlsx', sheet_name='Hoja1')

aviso_orig, mensaje_orig = impago_nomina_pension_desempleo(transacciones_nomina_orig_df)

print(aviso_orig)
print(mensaje_orig)


False
2020-8-5: El usuario aún no ha cobrado la nómina. Hay que avisarle
